In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import flopy
import shutil

In [ ]:
def hmean(x1,x2):
    return(2/np.sum((1/x1,1/x2)))

In [ ]:
dx = 10.
dy = 10.
nx = 40
b = 1.
kx = 1
h0_x0 = 10.
h0_xL = 5
R = 0.01

# fixed head both boundaries

In [ ]:
x = np.linspace(0,nx,nx)*dx
k = kx*np.ones_like(x)
Rx = R*dx*dy
h = np.zeros_like(x)
# add some heterogeneity
k[np.floor(nx/3).astype(int):] = 20
G = Rx

LHS = np.zeros((nx,nx))
RHS = np.ones(nx) * Rx
for i in range(1,nx-1): 
    D = - hmean(k[i-1],k[i]) * b * dy / (dx)
    F = - hmean(k[i+1],k[i]) * b * dy / (dx)
    E = -D - F
    LHS[i,i-1] = D
    LHS[i,i]=E
    LHS[i,i+1]= F
    RHS[i] = G

LHS[0,0] = 1
LHS[nx-1,nx-1] = 1

RHS[nx-1] = h0_xL
RHS[0]= h0_x0

In [ ]:
h=np.linalg.solve(LHS,RHS)

In [ ]:
sim = flopy.mf6.MFSimulation(sim_name='test',sim_ws='test', verbosity_level=0, exe_name='./mf6')
tdis = flopy.mf6.ModflowTdis(sim)
ims = flopy.mf6.ModflowIms(sim)
gwf = flopy.mf6.ModflowGwf(sim,modelname='test',save_flows=True)
dis = flopy.mf6.ModflowGwfdis(gwf, nrow=1, ncol=nx, delr=dy,delc=dx)
ic = flopy.mf6.ModflowGwfic(gwf,strt=7.5)
npf = flopy.mf6.ModflowGwfnpf(gwf, k=k)
chd = flopy.mf6.ModflowGwfchd(gwf, stress_period_data=[[(0,0,0),h0_x0],[(0,0,nx-1),h0_xL]])
rch = flopy.mf6.ModflowGwfrch(gwf,stress_period_data=[[(0,0,i),R] for i in range(nx)])
oc = flopy.mf6.ModflowGwfoc(gwf, budget_filerecord='test.bud', head_filerecord='test.hds',
                           saverecord=[('head','all'),('budget','all')])
shutil.copy2('mf6','test/mf6')

In [ ]:
k

In [ ]:
sim.write_simulation()
#shutil.copy2('mf6','test/.')
sim.run_simulation()
mfhead = gwf.output.head().get_data().flatten()

In [ ]:
plt.plot(x,h,'.-',c='blue', label='py')
plt.plot(x,mfhead,'x-',c='orange', label='mf6')
plt.legend()

# no flow left boundary

In [ ]:
h = np.zeros_like(x)
G = Rx

LHS = np.zeros((nx,nx))
RHS = np.ones(nx) * G
for i in range(1,nx-1): 
    D = - hmean(k[i-1],k[i]) * b * dy / (dx)
    F = - hmean(k[i+1],k[i]) * b * dy / (dx)
    E = -D - F
    LHS[i,i-1] = D
    LHS[i,i]=E
    LHS[i,i+1]= F
    RHS[i] = G

LHS[0,0] = hmean(k[1],k[0]) * b * dy / (dx) 
LHS[0,1] = -hmean(k[1],k[0]) * b * dy / (dx) 
LHS[nx-1,nx-1] = 1

RHS[nx-1] = h0_xL


In [ ]:
h=np.linalg.solve(LHS,RHS)

In [ ]:
sim = flopy.mf6.MFSimulation(sim_name='test',sim_ws='test', verbosity_level=0, exe_name='./mf6')
tdis = flopy.mf6.ModflowTdis(sim)
ims = flopy.mf6.ModflowIms(sim)
gwf = flopy.mf6.ModflowGwf(sim,modelname='test',save_flows=True)
dis = flopy.mf6.ModflowGwfdis(gwf, nrow=1, ncol=nx, delr=dy,delc=dx)
ic = flopy.mf6.ModflowGwfic(gwf,strt=7.5)
npf = flopy.mf6.ModflowGwfnpf(gwf, k=k)
chd = flopy.mf6.ModflowGwfchd(gwf, stress_period_data=[[(0,0,nx-1),h0_xL]])
rch = flopy.mf6.ModflowGwfrch(gwf,stress_period_data=[[(0,0,i),R] for i in range(nx)])
oc = flopy.mf6.ModflowGwfoc(gwf, budget_filerecord='test.bud', head_filerecord='test.hds',
                           saverecord=[('head','all'),('budget','all')])

In [ ]:
sim.write_simulation()
#shutil.copy2('mf6','test/.')
sim.run_simulation()
mfhead = gwf.output.head().get_data().flatten()

In [ ]:
plt.plot(x,h,'.-',c='blue', label='py')
plt.plot(x,mfhead,'x-',c='orange', label='mf6')
plt.legend()